In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import nbimporter
from AIBaseClass import ChessAI

# Aufgabe 02: Minimax (einfacher Materialwert)

Diese Klasse implementiert den Minimax-Algorithmus zur Berechnung des nächsten Zuges im Mittelspiel. Die Evaluierung eines Boards wird dabei durch Betrachtung des Materialwertes der Figuren realisiert.

Die Klasse wird um eine neue Konstante `DEPTH` erweitert. Diese gibt an, wie viele zukünftige Halbzüge bei der Berechnung des nächsten Zuges betrachtet werden sollen, das heißt, welche Höhe der entsprechende Baum aller möglichen nächsten Halbzüge aufweist. Die Konstante wird als Parameter übergeben und im Konstruktor gesetzt.

In [ ]:
import chess_custom as chess
import os

class Exercise02AI(ChessAI):
    """Chooses middle game moves using minimax algorithm and material values"""

    def __init__(self, player_name: str, search_depth: int = 3) -> None:
        super().__init__(player_name)
        self.DEPTH = int(os.environ.get("depth", search_depth))
        self.last_evaluation: int | None = None

    def reset(self) -> None:
        """Resets all internal variables"""
        super().reset()
        self.last_evaluation = None

## Evaluierungsfunktion

Die Evaluierungsfunktion `evaluate` nimmt ein Board als Argument und berechnet eine Ganzzahl als Maß dafür, ob die gegebene Stellung für den weißen Spieler eher zu einem Sieg (positiver Wert) oder zu einer Niederlage (negativer Wert) führt.

Diese Implementierung verwendet dafür den Materialwert (Tauschwert in Bauerneinheiten) nach folgender Tabelle:

| Figurname  | Materialwert |
|---|---|
| Bauer (pawn) | 1  |
| Springer (knight) | 3  |
| Läufer (bishop) | 3  |
| Turm (rook) | 5  |
| Dame (queen) | 9  |
| König (king) | 20000 |

Für die Berechnung des nächsten Schrittes stehen mehrere Funktionen zur Verfügung:

- `full_evaluate`
- `incremental_evaluate`
- `evaluate`

Die Funktion `full_evaluate` berechnet den Wert eines Boards von Grund auf neu und geht dabei wie folgt vor:

1. Es wird für jede Figur und für jede Farbe die Anzahl der Spielfiguren ermittelt.
2. Für jede Figur wird die Differenz der Anzahl pro Farbe gebildet ($\texttt{Weiß} - \texttt{Schwarz}$)
3. Jede Differenz wird mit dem figurenspezifischen Faktor multipliziert und zur Gesamtsumme addiert.

Die Funktion `incremental_evaluate` berechnet die Änderungen der Bewertung wenn der gegebene Zug auf dem übergebenen Board ausgeführt wird und gibt diese Änderung zurück. Hierfür werden die drei Fälle untersucht in denen sich der Wert verändert haben kann:

1. Ein En Passant Zug wurde durchgeführt
2. Eine Figur wurde geschlagen
3. Ein Bauer wurde in eine neue Figur umgewandelt

Die Funktion `evaluate` speichert den Rückgabewert der inkrementellen Berechnung im Cache und gibt ihn anschließend zurück.

In [ ]:
class Exercise02AI(Exercise02AI): # type: ignore
    MATERIAL_VALUES = {
        chess.PAWN: 1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK: 5,
        chess.QUEEN: 9,
        chess.KING: 20000
    }

    def full_evaluate(self, board: chess.Board) -> int:
        """Returns a full evaluation of the given board."""
        evaluation = 0
        for piece_type in chess.PIECE_TYPES:
            # Get the amount of pieces on the board per color
            amount_type_white   = len(board.pieces(piece_type,   chess.WHITE))
            amount_type_black   = len(board.pieces(piece_type,   chess.BLACK))
            # Get difference
            diff = amount_type_white - amount_type_black
            # Calculate material value
            evaluation += self.MATERIAL_VALUES[piece_type] * diff
        return evaluation

    def incremental_evaluate(self, board: chess.Board, last_move: chess.Move) -> int:
        """Returns an incrementally calculated evaluation of the given board."""
        change = 0

        if captured_piece_type := board.piece_type_at(last_move.to_square):
            change += self.MATERIAL_VALUES[captured_piece_type]

        if promotion_piece_type := last_move.promotion:
            change += self.MATERIAL_VALUES[promotion_piece_type] - self.MATERIAL_VALUES[chess.PAWN]
            
        elif captured_piece_type is None and board.is_en_passant(last_move):
            # If the first two cases didn't match, check for en passant move
            change += self.MATERIAL_VALUES[chess.PAWN]
        
        # Add to old evaluation if white to move, else subtract
        factor = +1 if board.turn else -1
        return change * factor

    def evaluate(self, board: chess.Board, last_move: chess.Move, last_evaluation: int) -> int:
        """Evaluates a given board. 
        Returns a positive value if white has a better material value than black."""
        return last_evaluation + self.incremental_evaluate(board, last_move)

## Fünffache Stellungswiederholung

Im Schach besagt die `fivefold repetition`-Regel, dass ein Spiel sofort als Remis gewertet wird, falls sich eine Stellung zum fünften Mal wiederholt. Zwei Stellungen gelten dabei als gleich g. d. w. die Boardbelegung (Farbe, Figurtyp, Position) und die möglichen Züge dieselben sind.
Die Prüfung auf eine Stellungswiederholung ist in der `chess`-Bibliothek sehr langsam, da im schlechtesten Fall das gesamte Spiel neu durchlaufen wird (siehe Dokumentation). Die Funktion `repetitions` führt diese Prüfung effizienter mithilfe der Boardbelegung durch und gibt die Anzahl der Wiederholungen zurück. Diese Prüfung wird nur innerhalb des Minimax-Algorithmus verwendet, nach jedem abgeschlossenen Zug findet ein zusätzlicher Test mithilfe der `chess`-Bibliothek statt.

## Minimax

Die rekursive `minimax`-Funktion betrachtet bei einem gegebenen Board alle möglichen Stellungen (Boards) nach einer gegebenen Anzahl von Halbzügen (Tiefe) und gibt die beste Evaluierung (`evaluate`-Funktion) für den aktuellen Spieler zurück.

Die Konstante `LIMIT` wird als obere (+) beziehungsweise untere (-) Schranke für den Vergleich innerhalb der ersten Iteration verwendet.

Folgende Fälle werden unterschieden:

1. Falls das Spiel beendet ist, werden die drei folgenden Möglichkeiten unterschieden:
    - Das Spiel ist ein Remis:
    In diesem Fall wird 0 als Wert zurückgegeben. Der Zug welcher zu diesem Zustand führt wird also nur gewählt, sofern jeder andere Zug zu einem für die Seite schlechteren Wert (und somit eher zu einer Niederlage) führt.
    - Weiß hat das Spiel gewonnen:
    Hier wird das positive Limit (`99999`) abzüglich der aktuellen Tiefe zurückgegeben, welches auf jeden Fall größer oder gleich dem Evaluierungswert aller anderen Züge ist. Weiß wird daher diesen (oder einen in dieser Hinsicht gleichwertigen) Zug wählen um das Spiel zu gewinnen, Schwarz versucht den Zug möglichst zu vermeiden.
    - Schwarz hat das Spiel gewonnen:
    Das negative Limit (`-99999`) abzüglich der aktuellen Tiefe wird zurückgegeben. Dieses ist auf jeden Fall kleiner oder gleich dem Evaluierungswert aller anderen Züge. Schwarz wird daher diesen (oder einen in dieser Hinsicht gleichwertigen) Zug wählen um das Spiel zu gewinnen, Weiß versucht den Zug möglichst zu vermeiden.
2. Falls die Tiefe Null ist, wird die Auswertung des aktuellen Boards zurückgegeben (Rekursionsende).
3. Falls die Tiefe Eins ist, wird eine vollständige Beurteilung des Boards erstellt und gespeichert. Auf diese Weise können auf der Tiefe 0 alle Bewertungen inkrementell erfolgen. In der Gesamtzahl sind damit ca. 97% aller Beurteilungen inkrementell.
4. Falls Weiß am Zug ist, wird jeder mögliche Zug rekursiv evaluiert und das Maximum der Ergebnisse zurückgegeben.
5. Falls Schwarz am Zug ist, wird jeder mögliche Zug rekursiv evaluiert und das Minimum der Ergebnisse zurückgegeben.

**Hinweis**:
Aufgrund der Prüfung auf die `fivefold repetition`-Regel (durch die Funktion `is_fivefold_repetition`) ist der hier implementierte Minimax-Algorithmus nicht nur von dem aktuellen Board (FEN), sondern auch von den bereits gespielten Zügen (Stack) abhängig.

In [ ]:
class Exercise02AI(Exercise02AI): # type: ignore
    LIMIT = 99999

    def minimax(self, board: chess.Board, depth: int, current_evaluation: int) -> tuple[int, list[chess.Move]]:
        """Searches the best value with given depth using minimax algorithm"""
        best_move = None
        
        if (is_checkmate := board.is_checkmate()) and not board.turn:
            # White has won the game
            evaluation = self.LIMIT - (self.DEPTH - depth)
            return evaluation, None
        elif is_checkmate and board.turn:
            # Black has won the game
            evaluation = -self.LIMIT + (self.DEPTH - depth)
            return evaluation, None
        # Check if game is over
        elif board.is_insufficient_material() or not board.legal_moves or board.is_fifty_moves() or board.is_repetition(5):
            # Game is very likely a draw
            return 0, None

        # Recursion abort case
        if depth == 0:
            return current_evaluation, None

        # White to play (positive numbers are good)
        if board.turn:
            maxEvaluation = -self.LIMIT
            for move in board.legal_moves:
                next_move_evaluation = self.evaluate(board, move, current_evaluation)
                board.push(move)
                evaluation, _ = self.minimax(board, depth - 1, next_move_evaluation)
                board.pop()
                if depth == self.DEPTH and evaluation >= maxEvaluation:
                    best_move = move
                maxEvaluation = max(maxEvaluation, evaluation)
            return maxEvaluation, best_move

        # Black to play (negative numbers are good)
        else:
            minEvaluation = self.LIMIT
            for move in board.legal_moves:
                next_move_evaluation = self.evaluate(board, move, current_evaluation)
                board.push(move)
                evaluation, _ = self.minimax(board, depth - 1, next_move_evaluation)
                board.pop()
                if depth == self.DEPTH and evaluation <= minEvaluation:
                    best_move = move
                minEvaluation = min(minEvaluation, evaluation)
            return minEvaluation, best_move

## Berechnung des besten Zuges

Die Funktion `get_next_middle_game_move` berechnet auf einem gegebenen Board den besten nächsten Zug. Dabei wird jeder mögliche Zug mithilfe der `minimax`-Funktion in der gewünschten Tiefe evaluiert und der Zug mit der besten Bewertung zurückgegeben.

In [ ]:
class Exercise02AI(Exercise02AI): # type: ignore
    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move | None:
        """Gets the best next move"""
        if self.last_evaluation is None:  # type: ignore
            current_evaluation = self.full_evaluate(board)
        else:
            last_move = board.pop()
            # Get current evaluation (after opponent move)
            current_evaluation = self.evaluate(board, last_move, self.last_evaluation)  # type: ignore
            board.push(last_move)

        # Call minimax and get best move
        _, best_move = self.minimax(board, self.DEPTH, current_evaluation)
        # Update last evaluation (after player move)
        assert best_move, f"""
        Best move is None with fen '{board.fen()}' at player {type(self).__name__}! 
        depth: {self.DEPTH}, last_eval: {self.last_evaluation}, current_evaluation: {current_evaluation},
        is_king_engame: {getattr(self, 'is_king_endgame', "N/A")}, move_stack: {board.move_stack}
        """
        self.last_evaluation = self.evaluate(board, best_move, current_evaluation)
        return best_move

## Debugging Bereich

Die folgenden Zellen enthalten Code zum Testen der oben implementierten Funktionen.

In [ ]:
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
player = Exercise02AI("Testplayer", 3)

In [ ]:
last_eval = player.evaluate(board)
last_eval

In [ ]:
board.push(chess.Move.from_uci("d4c3"))

In [ ]:
print(player.evaluate(board, last_eval))
print(player.evaluate(board))

In [ ]:
from time import time
def time_full_evals(num):
    board = chess.Board()
    player = Exercise02AI("Testplayer", 3)
    last_eval = player.evaluate(board)
    start = time()
    for _ in range(num):
        move = list(board.legal_moves)[0]
        board.push(move)
        last_eval = player.evaluate(board)
    end = time()
    return end - start

In [ ]:
def time_inc_evals(num):
    board = chess.Board()
    player = Exercise02AI("Testplayer", 3)
    last_eval = player.evaluate(board)
    start = time()
    for _ in range(num):
        move = list(board.legal_moves)[0]
        board.push(move)
        last_eval = player.evaluate(board, last_eval)
    end = time()
    return end - start

In [ ]:
time_full_evals(10000)

In [ ]:
time_inc_evals(10000)

In [ ]:
# Debugging function
from IPython.display import display
def display_path(board_orig, move, depth):
    """Displays the move path to the best evaluation."""
    board = deepcopy(board_orig)
    display(board)
    board.push(move)
    display(board)
    for i in range(depth, 0, -1):
        move = Exercise02AI.path_history[str(board) + str(i)]
        board.push(move)
        display(board)

In [ ]:
# Test evaluate
board = chess.Board("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
board.set_fen("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
board.push(list(board.legal_moves)[0])
player = Exercise02AI("Testplayer", 3)

player.evaluate(board)

In [ ]:
# Test evaluate
board = chess.Board("4k3/5p2/8/6P1/8/8/8/4K3 b - - 0 1")
board.push(chess.Move.from_uci('f7f5')) # two square pawn move
board.push(chess.Move.from_uci('g5f6')) # en passant capture
player = Exercise02AI("Testplayer", 3)
player.evaluate(board)

In [ ]:
%%time
# Test get_next_middle_game_move
DEPTH = 3
board = chess.Board()
board.set_fen("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
player = Exercise02AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player.get_next_middle_game_move(board)
print(move)
#display_path(board, move, DEPTH)